In [6]:
## Imports

import csv
import pandas as pd 

In [3]:
# Setup Twilio interface
import os
from twilio.rest import Client

account_sid = os.environ['TWILIO_ACCOUNT_SID'] = "AC9077560e954b4c3f7d41e071d7ce11fa"
auth_token = os.environ['TWILIO_AUTH_TOKEN'] = "29c0fede2bae7fb2f0233010be12ece6"
client = Client(account_sid, auth_token)

In [4]:
## Setup

# Paths
PATH = 'C:/Users/Sirpimmon/Desktop/hackathon22s/plantbuddy/'
conditions_path = "outputs/conditions.csv"
temps_path = "outputs/plant_temp_ranges.csv"
readings = "readings/reading1.csv" # For when it's using the renamed sheet
# readings = "readings/out.csv" # For when it's read directly from the arduino

# Time interval
unit = 0 # 0 for seconds, 1 for minutes
interval = 5 # 5 for minutes, 20 for seconds for demonstration

In [7]:
## Make dictionary of plants

filename = PATH + temps_path
file = open(filename)

csvreader = csv.reader(file)
plant_dict = {}

header = []
header = next(csvreader)

for row in csvreader:
    plant_dict[row[0]] = (float(row[2]), float(row[1]))

file.close()

In [9]:
# Read conditions csv

filename = PATH + conditions_path
conditions = pd.read_csv(filename, delimiter = ',', encoding="utf-8-sig")

In [10]:
# Ideal plant conditions from file

max_press = conditions._get_value(0, "max_press")
min_press = conditions._get_value(0, "min_press")
max_humid = conditions._get_value(0, "max_humid")
min_humid = conditions._get_value(0, "min_humid")
low_light_threshold = conditions._get_value(0, "low_light_threshold")
high_light_threshold = conditions._get_value(0, "high_light_threshold")

In [11]:
## Take input

print("What type of plant do you have?")
x = input().lower()

# If it's not in the American plants list, then it's assumed it's a houseplant
if x not in plant_dict:
    plant_dict[x] = (60, 80)

What type of plant do you have?
iris


In [19]:
## Read values from arduino
## Should only be one line
def readvals(filename):
    global light
    global temp
    global time
    global humidity
    global pressure
    
    
    file = open(filename)

    csvreader = csv.reader(file)

    header = []
    header = next(csvreader)

    for row in csvreader:
        if unit == 0:
            time = int(row[0][3:5])
        else:
            time = int(row[0][0:2])
        light = int(row[1])
        temp = (float(row[2])*1.8) + 32
        humidity = float(row[3])
        pressure = float(row[4])

    file.close()

filename = PATH + readings    
readvals(filename)

In [16]:
filename

'C:/Users/Sirpimmon/Desktop/hackathon22s/plantbuddy/readings/reading1.csv'

In [20]:
## Check levels

# 0 for good level
# 1 for too low
# 2 for too high

light_lvl = 0
temp_lvl = 0
humidity_lvl = 0
pressure_lvl = 0

if light < low_light_threshold:
    light_lvl = 1
elif light > high_light_threshold:
    light_lvl = 2

if temp < plant_dict[x][0]:
    temp_lvl = 1
elif temp > plant_dict[x][1]:
    temp_lvl = 2

if humidity < min_humid:
    humidity_lvl = 1
elif humidity > max_humid:
    humidity_lvl = 2
    
if pressure < min_press:
    pressure_lvl = 1
elif pressure > max_press:
    pressure_lvl = 2

In [22]:
## Reports

prev_time_recorded = 0
output = [light_lvl, temp_lvl, plant_dict[x], humidity_lvl, pressure_lvl]

# If it's time to report a status update
if time%interval == 0:
    prev_time_recorded = time
    
    print(output)
    

output


[1, 0, (60.0, 80.0), 1, 0]

In [24]:
## Msg generator

# The print is for if the interface is not enabled
# The saved variable is for the texting


msg = ""

if light_lvl == 1:
    print("I need more light. Please move me into the sun!")
    msg = msg + "Your plant needs more light, "
elif light_lvl == 2:
    print("I need less light. Please move me into the shade!")
    msg = msg + "Your plant needs less light, "
else:
    print("This much light is good!")

if temp_lvl == 1:
    print("I'm cold! Raise the temperature to " + str(plant_dict[x][0]) + " degrees.")
    msg = msg + "to be warmed up to " + str(plant_dict[x][0]) + " degrees, "
elif temp_lvl == 2:
    print("I'm hot! Lower the temperature to " + str(plant_dict[x][1]) + " degrees.")
    msg = msg + "to be cooled down to " + str(plant_dict[x][1]) + " degrees, "
else:
    print("This temperature is good!")
    
if humidity_lvl == 1:
    print("I'm thirsty! Please water me.")
    msg = msg + "and to be watered. "
elif humidity_lvl == 2:
    print("I'm too wet! Please stop watering me.")
    msg = msg + "and to stop being watered. "
else:
    print("This is a perfect amount of water!")
    
if pressure_lvl == 1:
    print("This isn't enough air pressure!")
    msg = msg + "And more pressure."
elif pressure_lvl == 2:
    print("This is too much air pressure!")
    msg = msg + "And less pressure."
else:
    print("This amount of pressure is good!")
    
if temp_lvl + pressure_lvl + humidity_lvl + light_lvl == 0:
    msg = "Everything is great!"
print(msg)

I need more light. Please move me into the sun!
This temperature is good!
I'm thirsty! Please water me.
This amount of pressure is good!
Your plant needs more light, and to be watered. 


In [25]:
## Send text message


message = client.messages \
    .create(
         body=msg,
         from_='+18438060190',
         to='+15018301647'
     )

print(message.sid)

SMdc2dd7d52aa545d9bda8f13ed63507f7
